<a href="https://colab.research.google.com/github/tharudew7/Training-Image-Classification-Deep-Learning-Models/blob/main/plantTesting01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tensorflow-gpu==2.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 458.4/458.4 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 91.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 99.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 kB 18.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for clang: filename=clang-5.0-py3-none-any.whl size=30692 sha256=a79a843b618e391af1ad4aff50f56628b1fbd536fb6a2f847f9d1de063e5bc14
  Stored in directory: /root/.cache/pip/wheels/3a/ce/7a/27094f689461801c934296d07078773603663dfcaca63bb064
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4845 sha256=1a094745fd5e61cb680257bafa4e14cad

In [ ]:
!nvidia-smi

Thu Apr 13 17:04:47 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    27W /  70W |    387MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten

In [ ]:
from tensorflow.keras.models import Model

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [ ]:
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input


In [ ]:
from tensorflow.keras.preprocessing import image

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img

In [ ]:
from tensorflow.keras.models import Sequential

In [ ]:
import numpy as np

In [ ]:
from glob import glob

In [41]:
#re size all the images to this
IMAGE_SIZE = [224,224]

train_path = '/content/drive/MyDrive/datasets/train'
valid_path = '/content/drive/MyDrive/datasets/valid'

In [42]:
#Import the Inception V3 library as shown below and add preprocessing layer to the front of VGG
#Here we will be usingg imagenet weights

inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet',include_top=False)

In [43]:
#Don't train existing weights
for layer in inception.layers:
    layer.trainable = False

In [44]:
#useful for getting number of output classes
folders = glob('/content/drive/MyDrive/datasets/train/*')

In [45]:
#our layers - you can add more if you want
x = Flatten()(inception.output)

In [46]:
prediction = Dense(len(folders), activation='softmax')(x)

#create a model object
model = Model(inputs=inception.input, outputs=prediction)

In [47]:
#view the structure of the model
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_97 (Conv2D)             (None, 111, 111, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_94 (BatchN  (None, 111, 111, 32  96         ['conv2d_97[0][0]']              
 ormalization)                  )                                                           

In [48]:
#tell the model what cost and optimization mothod to use

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
    
)

In [53]:
#Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range= 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [54]:
#Make sure you provide the same target size as initailized for the image size

training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/datasets/train',
                                                 target_size = (224,224),
                                                 batch_size = 16,
                                                 class_mode = 'categorical')

Found 1900 images belonging to 10 classes.


In [55]:
#fit the model
#Run the cell. It will take some time to execute

r = model.fit_generator(
    training_set,
    validation_data=test_set,
    epochs=10,
    steps_per_epoch=len(training_set),
    validation_steps=len(test_set)
)

<ipython-input-55-b67a941b40bf>:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/10
119/119 [==============================] - 396s 3s/step - loss: 7.2146 - accuracy: 0.4395
Epoch 2/10
119/119 [==============================] - 28s 234ms/step - loss: 3.0115 - accuracy: 0.6679
Epoch 3/10
119/119 [==============================] - 30s 250ms/step - loss: 1.9580 - accuracy: 0.7605
Epoch 4/10
119/119 [==============================] - 29s 240ms/step - loss: 2.5354 - accuracy: 0.7479
Epoch 5/10
119/119 [==============================] - 29s 242ms/step - loss: 1.8571 - accuracy: 0.8047
Epoch 6/10
119/119 [==============================] - 31s 259ms/step - loss: 1.9558 - accuracy: 0.8047
Epoch 7/10
119/119 [==============================] - 28s 237ms/step - loss: 1.5145 - accuracy: 0.8395
Epoch 8/10
119/119 [==============================] - 30s 253ms/step - loss: 1.7096 - accuracy: 0.8474
Epoch 9/10
119/119 [==============================] - 29s 240ms/step - loss: 1.6139 - accuracy: 0.8437
Epoch 10/10
119/119 [==============================] - 28s 236ms/step - los